In [8]:
import gpxpy
import gpxpy.gpx
import pandas as pd
from pathlib import Path
from typing import List, Tuple
from haversine import haversine, Unit
import folium
from folium.plugins import MarkerCluster


# Step 1: 刪除時間 + 匯出 CSV
def remove_time_and_export_csv(gpx_file: str) -> Tuple[pd.DataFrame, gpxpy.gpx.GPX]:
    with open(gpx_file, "r", encoding="utf-8") as f:
        gpx = gpxpy.parse(f)

    data = []
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                data.append(
                    {
                        "latitude": point.latitude,
                        "longitude": point.longitude,
                        "elevation": point.elevation,
                    }
                )
                point.time = None  # 去除時間

    df = pd.DataFrame(data)
    csv_name = Path(gpx_file).stem + ".csv"
    df.to_csv(csv_name, index=False)
    print(f"CSV 匯出完成：{csv_name}")
    return df, gpx


# Step 2: 讀取通訊點
def load_communication_points(txt_file: str) -> List[Tuple[str, float, float, float]]:
    df = pd.read_csv(txt_file, sep="\t")
    points = []
    for _, row in df.iterrows():
        lat = float(str(row["緯度"]).replace("°", "").strip())
        lon = float(str(row["經度"]).replace("°", "").strip())
        ele = float(str(row["海拔（約）"]).replace("m", "").strip())
        label = f"{row['步道名稱']} {row['路標指示']}"
        points.append((label, lat, lon, ele))
    return points


# Step 3: 插入通訊點
def insert_points(
    gpx: gpxpy.gpx.GPX, comm_pts: List[Tuple[str, float, float, float]]
) -> gpxpy.gpx.GPX:
    for label, lat, lon, ele in comm_pts:
        closest_dist = float("inf")
        insert_idx = None
        segment = gpx.tracks[0].segments[0]
        points = segment.points

        for i in range(len(points) - 1):
            d1 = haversine(
                (points[i].latitude, points[i].longitude), (lat, lon), unit=Unit.METERS
            )
            d2 = haversine(
                (points[i + 1].latitude, points[i + 1].longitude),
                (lat, lon),
                unit=Unit.METERS,
            )
            if d1 + d2 < closest_dist:
                closest_dist = d1 + d2
                insert_idx = i + 1

        new_point = gpxpy.gpx.GPXTrackPoint(latitude=lat, longitude=lon, elevation=ele)
        segment.points.insert(insert_idx, new_point)
        print(f"已插入通訊點：{label}")
    return gpx


# Step 4: 切割單一路線（由第一至最後通訊點之間）
def split_single_segment(
    gpx: gpxpy.gpx.GPX,
    start_latlon: Tuple[float, float],
    end_latlon: Tuple[float, float],
) -> gpxpy.gpx.GPX:
    points = gpx.tracks[0].segments[0].points

    def find_idx(target: Tuple[float, float]) -> int:
        return min(
            range(len(points)),
            key=lambda i: haversine(
                (points[i].latitude, points[i].longitude), target, unit=Unit.METERS
            ),
        )

    idx_start = find_idx(start_latlon)
    idx_end = find_idx(end_latlon)
    if idx_start > idx_end:
        idx_start, idx_end = idx_end, idx_start

    new_gpx = gpxpy.gpx.GPX()
    new_track = gpxpy.gpx.GPXTrack()
    new_segment = gpxpy.gpx.GPXTrackSegment()
    new_segment.points = points[idx_start : idx_end + 1]
    new_track.segments.append(new_segment)
    new_gpx.tracks.append(new_track)

    print(f"切割完成：索引 {idx_start} 至 {idx_end}")
    return new_gpx


# import folium
from folium.plugins import MarkerCluster


def visualize_with_original_and_cut(
    original_gpx: gpxpy.gpx.GPX,
    sliced_gpx: gpxpy.gpx.GPX,
    comm_pts: List[Tuple[str, float, float, float]],
    output_html: str,
):
    # 初始地圖中心
    first_point = original_gpx.tracks[0].segments[0].points[0]
    m = folium.Map(
        location=[first_point.latitude, first_point.longitude], zoom_start=14
    )

    # 圖層群組
    original_layer = folium.FeatureGroup(name="原始路線", show=True)
    cut_layer = folium.FeatureGroup(name="切割後路線", show=True)
    comm_layer = folium.FeatureGroup(name="通訊點", show=True)

    # 原始路線 PolyLine
    original_coords = [
        (pt.latitude, pt.longitude) for pt in original_gpx.tracks[0].segments[0].points
    ]
    folium.PolyLine(
        original_coords, color="blue", weight=4, opacity=0.6, tooltip="原始路線"
    ).add_to(original_layer)

    # 切割後路線 PolyLine
    cut_coords = [
        (pt.latitude, pt.longitude) for pt in sliced_gpx.tracks[0].segments[0].points
    ]
    folium.PolyLine(
        cut_coords, color="red", weight=4, opacity=0.8, tooltip="切割後路線"
    ).add_to(cut_layer)

    # 通訊點 Marker
    cluster = MarkerCluster(name="通訊點群組").add_to(comm_layer)
    for label, lat, lon, ele in comm_pts:
        folium.Marker(
            [lat, lon],
            popup=f"{label}<br>海拔：約{ele} m",
            icon=folium.Icon(color="purple", icon="info-sign"),
        ).add_to(cluster)

    # 加入各層至地圖
    original_layer.add_to(m)
    cut_layer.add_to(m)
    comm_layer.add_to(m)

    # 圖層控制按鈕
    folium.LayerControl().add_to(m)

    # 儲存HTML
    m.save(output_html)
    print(f"✅ 地圖已儲存：{output_html}")


# --- 主程式 ---
if __name__ == "__main__":
    gpx_path = "標準桃山.gpx"
    txt_path = "communication_point.txt"

    # 1. 時間刪除與匯出 CSV
    df, gpx_obj = remove_time_and_export_csv(gpx_path)

    # 2. 載入與插入通訊點
    comm_pts = load_communication_points(txt_path)
    gpx_updated = insert_points(gpx_obj, comm_pts)

    # 3. 切出單一段路線
    start_latlon = (comm_pts[0][1], comm_pts[0][2])
    end_latlon = (comm_pts[-1][1], comm_pts[-1][2])
    sliced_gpx = split_single_segment(gpx_updated, start_latlon, end_latlon)

    # 4. 儲存 GPX
    with open("single_segment.gpx", "w", encoding="utf-8") as f:
        f.write(sliced_gpx.to_xml())
    print("儲存單一路線GPX：single_segment.gpx")

    # 5. 顯示完整地圖（含圖層控制）
    visualize_with_original_and_cut(
        gpx_updated, sliced_gpx, comm_pts, "route_map_with_control.html"
    )

FileNotFoundError: [Errno 2] No such file or directory: '標準桃山.gpx'

In [6]:
import gpxpy
import gpxpy.gpx
import pandas as pd
from pathlib import Path
from typing import List, Tuple
from haversine import haversine, Unit
import folium
from folium.plugins import MarkerCluster

# Step 1: 去除時間
def remove_time(gpx_file: str) -> gpxpy.gpx.GPX:
    with open(gpx_file, "r", encoding="utf-8") as f:
        gpx = gpxpy.parse(f)
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                point.time = None
    print(f"✅ 已移除時間：{gpx_file}")
    return gpx

# Step 2: 讀取通訊點
def load_communication_points(txt_file: str) -> List[Tuple[str, float, float, float]]:
    df = pd.read_csv(txt_file, sep="\t")
    points = []
    for _, row in df.iterrows():
        lat = float(str(row["緯度"]).replace("°", "").strip())
        lon = float(str(row["經度"]).replace("°", "").strip())
        ele = float(str(row["海拔（約）"]).replace("m", "").strip())
        label = f"{row['步道名稱']} {row['路標指示']}"
        points.append((label, lat, lon, ele))
    return points

# Step 2-2: 插入通訊點
def insert_points(gpx: gpxpy.gpx.GPX, comm_pts: List[Tuple[str, float, float, float]]) -> gpxpy.gpx.GPX:
    segment = gpx.tracks[0].segments[0]
    points = segment.points
    for label, lat, lon, ele in comm_pts:
        closest_dist = float("inf")
        insert_idx = None
        for i in range(len(points) - 1):
            d1 = haversine((points[i].latitude, points[i].longitude), (lat, lon), unit=Unit.METERS)
            d2 = haversine((points[i + 1].latitude, points[i + 1].longitude), (lat, lon), unit=Unit.METERS)
            if d1 + d2 < closest_dist:
                closest_dist = d1 + d2
                insert_idx = i + 1
        new_point = gpxpy.gpx.GPXTrackPoint(latitude=lat, longitude=lon, elevation=ele)
        segment.points.insert(insert_idx, new_point)
        print(f"已插入通訊點：{label}")
    return gpx

# Step 3: 切割通訊點之間路線
def split_single_segment(gpx: gpxpy.gpx.GPX, start_latlon: Tuple[float, float], end_latlon: Tuple[float, float]) -> gpxpy.gpx.GPX:
    points = gpx.tracks[0].segments[0].points
    def find_idx(target: Tuple[float, float]) -> int:
        return min(range(len(points)), key=lambda i: haversine((points[i].latitude, points[i].longitude), target, unit=Unit.METERS))
    idx_start = find_idx(start_latlon)
    idx_end = find_idx(end_latlon)
    if idx_start > idx_end:
        idx_start, idx_end = idx_end, idx_start
    new_gpx = gpxpy.gpx.GPX()
    new_track = gpxpy.gpx.GPXTrack()
    new_segment = gpxpy.gpx.GPXTrackSegment()
    new_segment.points = points[idx_start: idx_end + 1]
    new_track.segments.append(new_segment)
    new_gpx.tracks.append(new_track)
    print(f"切割完成：索引 {idx_start} 至 {idx_end}")
    return new_gpx

# Step 4: 匯出 GPX 與 CSV
def export_gpx_and_csv(gpx: gpxpy.gpx.GPX, gpx_name: str, csv_name: str):
    # 儲存 GPX
    with open(gpx_name, "w", encoding="utf-8") as f:
        f.write(gpx.to_xml())
    print(f"✅ 儲存 GPX：{gpx_name}")

    # 匯出 CSV
    data = []
    for pt in gpx.tracks[0].segments[0].points:
        data.append({"latitude": pt.latitude, "longitude": pt.longitude, "elevation": pt.elevation})
    df = pd.DataFrame(data)
    df.to_csv(csv_name, index=False)
    print(f"✅ 匯出 CSV：{csv_name}")

# Step 5: 顯示地圖（含Checkbox控制）
# Step 5: 顯示地圖（使用內建 LayerControl）
def visualize_with_checkbox(original_gpx, sliced_gpx, comm_pts, output_html):
    # 找到地圖中心點
    if original_gpx.tracks[0].segments[0].points:
        first_point = original_gpx.tracks[0].segments[0].points[0]
        map_location = [first_point.latitude, first_point.longitude]
    else:
        map_location = [23.5, 121] # 如果沒有點，預設台灣中心

    m = folium.Map(location=map_location, zoom_start=14)

    # --- 建立圖層 ---
    # 為每個圖層建立 FeatureGroup，並給予在控制器上顯示的名稱
    original_layer = folium.FeatureGroup(name="原始路線", show=True)
    cut_layer = folium.FeatureGroup(name="切割後路線", show=True)
    comm_layer = folium.FeatureGroup(name="通訊點", show=True)

    # --- 將資料加入對應圖層 ---
    # 原始路線
    original_coords = [(pt.latitude, pt.longitude) for pt in original_gpx.tracks[0].segments[0].points]
    folium.PolyLine(original_coords, color="blue", weight=4, opacity=0.6).add_to(original_layer)

    # 切割後路線
    cut_coords = [(pt.latitude, pt.longitude) for pt in sliced_gpx.tracks[0].segments[0].points]
    folium.PolyLine(cut_coords, color="red", weight=4, opacity=0.8).add_to(cut_layer)

    # 通訊點 (使用 MarkerCluster)
    cluster = MarkerCluster().add_to(comm_layer)
    for label, lat, lon, ele in comm_pts:
        folium.Marker(
            [lat, lon], 
            popup=f"{label}<br>海拔：約{ele} m", 
            icon=folium.Icon(color="purple")
        ).add_to(cluster)

    # --- 將圖層加入地圖 ---
    original_layer.add_to(m)
    cut_layer.add_to(m)
    comm_layer.add_to(m)

    # --- 加入圖層控制器 (這會自動產生 Checkbox) ---
    folium.LayerControl().add_to(m)

    m.save(output_html)
    print(f"✅ 地圖已儲存：{output_html}")



if __name__ == "__main__":
    # 🔹設定資料夾路徑
    gpx_folder = Path("hikingbook_route")
    txt_folder = Path("txt_data")
    gpx_output_folder = Path("gpx_data")
    csv_output_folder = Path("csv_data")
    html_output_folder = Path("Html_data")

    # 🔹建立輸出資料夾（若不存在）
    gpx_output_folder.mkdir(exist_ok=True)
    csv_output_folder.mkdir(exist_ok=True)
    html_output_folder.mkdir(exist_ok=True)

    # 🔹迴圈處理每個 GPX 檔案
    for gpx_file in gpx_folder.glob("*.gpx"):
        base_name = gpx_file.stem  # 檔名不含副檔名
        txt_file = txt_folder / f"{base_name}.txt"

        if not txt_file.exists():
            print(f"⚠️ 找不到對應的 TXT 檔案：{txt_file.name}，略過此 GPX")
            continue

        print(f"\n🔸開始處理：{gpx_file.name}")

        # 1. 去除時間
        gpx_obj = remove_time(str(gpx_file))

        # 2. 插入通訊點
        comm_pts = load_communication_points(str(txt_file))
        gpx_with_pts = insert_points(gpx_obj, comm_pts)

        # 3. 切出路段
        start_latlon = (comm_pts[0][1], comm_pts[0][2])
        end_latlon = (comm_pts[-1][1], comm_pts[-1][2])
        sliced_gpx = split_single_segment(gpx_with_pts, start_latlon, end_latlon)

        # 4. 匯出 GPX + CSV（命名用原始 GPX 名）
        gpx_outfile = gpx_output_folder / f"{base_name}.gpx"
        csv_outfile = csv_output_folder / f"{base_name}.csv"
        export_gpx_and_csv(sliced_gpx, str(gpx_outfile), str(csv_outfile))

        # 5. 儲存地圖 HTML（用原始 GPX 檔名）
        html_outfile = html_output_folder / f"{base_name}.html"
        visualize_with_checkbox(gpx_with_pts, sliced_gpx, comm_pts, str(html_outfile))

    print("\n✅ 批次處理完成")



✅ 批次處理完成


In [7]:
import gpxpy
import gpxpy.gpx
import pandas as pd
from pathlib import Path
from typing import List, Tuple
from haversine import haversine, Unit
import folium
from folium.plugins import MarkerCluster
import os

# Step 1: 去除時間
def remove_time(gpx_file: str) -> gpxpy.gpx.GPX:
    """從 GPX 檔案中移除所有軌跡點的時間戳記。"""
    with open(gpx_file, "r", encoding="utf-8") as f:
        gpx = gpxpy.parse(f)
    for track in gpx.tracks:
        for segment in track.segments:
            for point in segment.points:
                point.time = None
    print(f"    - 已移除時間戳記")
    return gpx

# Step 2: 讀取通訊點
def load_communication_points(txt_file: str) -> List[Tuple[str, float, float, float]]:
    """從 TXT 檔案讀取通訊點資料。"""
    df = pd.read_csv(txt_file, sep="\t")
    points = []
    for _, row in df.iterrows():
        try:
            lat = float(str(row["緯度"]).replace("°", "").strip())
            lon = float(str(row["經度"]).replace("°", "").strip())
            ele = float(str(row["海拔（約）"]).replace("m", "").strip())
            label = f"{row['步道名稱']} {row['路標指示']}"
            points.append((label, lat, lon, ele))
        except (ValueError, KeyError) as e:
            print(f"      - ⚠️ 讀取通訊點時發生錯誤，請檢查 TXT 檔案格式：{e}")
            continue
    print(f"    - 成功讀取 {len(points)} 個通訊點")
    return points

# Step 2-2: 插入通訊點
def insert_points(gpx: gpxpy.gpx.GPX, comm_pts: List[Tuple[str, float, float, float]]) -> gpxpy.gpx.GPX:
    """將通訊點插入到 GPX 軌跡中最接近的位置。"""
    if not gpx.tracks or not gpx.tracks[0].segments:
        print("    - ⚠️ GPX 檔案中沒有軌跡資料，無法插入通訊點。")
        return gpx
        
    segment = gpx.tracks[0].segments[0]
    points = segment.points
    
    for label, lat, lon, ele in comm_pts:
        closest_dist = float("inf")
        insert_idx = None
        # 尋找最佳插入點
        for i in range(len(points) - 1):
            p1 = (points[i].latitude, points[i].longitude)
            p2 = (points[i + 1].latitude, points[i + 1].longitude)
            comm_p = (lat, lon)
            
            # 計算點到線段的距離（簡化為點到兩端點距離和）
            dist_sum = haversine(p1, comm_p, unit=Unit.METERS) + haversine(p2, comm_p, unit=Unit.METERS)
            
            if dist_sum < closest_dist:
                closest_dist = dist_sum
                insert_idx = i + 1
        
        if insert_idx is not None:
            new_point = gpxpy.gpx.GPXTrackPoint(latitude=lat, longitude=lon, elevation=ele)
            segment.points.insert(insert_idx, new_point)
            # print(f"    - 已插入通訊點：{label}")
    print(f"    - 已將所有通訊點插入軌跡")
    return gpx

# Step 3: 切割通訊點之間路線
def split_single_segment(gpx: gpxpy.gpx.GPX, start_latlon: Tuple[float, float], end_latlon: Tuple[float, float]) -> gpxpy.gpx.GPX:
    """根據起點和終點的經緯度切割 GPX 軌跡。"""
    new_gpx = gpxpy.gpx.GPX()
    new_track = gpxpy.gpx.GPXTrack()
    new_segment = gpxpy.gpx.GPXTrackSegment()
    new_gpx.tracks.append(new_track)
    new_track.segments.append(new_segment)

    if not gpx.tracks or not gpx.tracks[0].segments:
        print("    - ⚠️ GPX 檔案中沒有軌跡資料，無法進行切割。")
        return new_gpx

    points = gpx.tracks[0].segments[0].points
    
    def find_idx(target: Tuple[float, float]) -> int:
        return min(range(len(points)), key=lambda i: haversine((points[i].latitude, points[i].longitude), target, unit=Unit.METERS))
        
    idx_start = find_idx(start_latlon)
    idx_end = find_idx(end_latlon)
    
    if idx_start > idx_end:
        idx_start, idx_end = idx_end, idx_start
        
    new_segment.points = points[idx_start : idx_end + 1]
    print(f"    - 路線切割完成：從索引 {idx_start} 至 {idx_end}")
    return new_gpx

# Step 4: 匯出 GPX 與 CSV
def export_gpx_and_csv(gpx: gpxpy.gpx.GPX, gpx_name: str, csv_name: str):
    """將處理後的 GPX 物件儲存為 GPX 和 CSV 檔案。"""
    # 儲存 GPX
    with open(gpx_name, "w", encoding="utf-8") as f:
        f.write(gpx.to_xml())
    print(f"    - ✅ 儲存 GPX：{gpx_name}")

    # 匯出 CSV
    data = []
    if gpx.tracks and gpx.tracks[0].segments:
        for pt in gpx.tracks[0].segments[0].points:
            data.append({"latitude": pt.latitude, "longitude": pt.longitude, "elevation": pt.elevation})
    df = pd.DataFrame(data)
    df.to_csv(csv_name, index=False)
    print(f"    - ✅ 匯出 CSV：{csv_name}")

# Step 5: 顯示地圖（使用內建 LayerControl）
def visualize_with_checkbox(original_gpx, sliced_gpx, comm_pts, output_html):
    """產生包含原始路徑、切割路徑和通訊點的互動式地圖。"""
    # 找到地圖中心點
    if original_gpx.tracks and original_gpx.tracks[0].segments and original_gpx.tracks[0].segments[0].points:
        first_point = original_gpx.tracks[0].segments[0].points[0]
        map_location = [first_point.latitude, first_point.longitude]
    else:
        map_location = [23.5, 121] # 如果沒有點，預設台灣中心

    m = folium.Map(location=map_location, zoom_start=14)

    # --- 建立圖層 ---
    original_layer = folium.FeatureGroup(name="原始路線", show=True)
    cut_layer = folium.FeatureGroup(name="切割後路線", show=True)
    comm_layer = folium.FeatureGroup(name="通訊點", show=True)

    # --- 將資料加入對應圖層 ---
    # 原始路線
    if original_gpx.tracks and original_gpx.tracks[0].segments:
        original_coords = [(pt.latitude, pt.longitude) for pt in original_gpx.tracks[0].segments[0].points]
        folium.PolyLine(original_coords, color="blue", weight=4, opacity=0.6).add_to(original_layer)

    # 切割後路線
    if sliced_gpx.tracks and sliced_gpx.tracks[0].segments:
        cut_coords = [(pt.latitude, pt.longitude) for pt in sliced_gpx.tracks[0].segments[0].points]
        folium.PolyLine(cut_coords, color="red", weight=4, opacity=0.8).add_to(cut_layer)

    # 通訊點 (使用 MarkerCluster)
    cluster = MarkerCluster().add_to(comm_layer)
    for label, lat, lon, ele in comm_pts:
        folium.Marker(
            [lat, lon], 
            popup=f"{label}<br>海拔：約{ele} m", 
            icon=folium.Icon(color="purple")
        ).add_to(cluster)

    # --- 將圖層加入地圖 ---
    original_layer.add_to(m)
    cut_layer.add_to(m)
    comm_layer.add_to(m)

    # --- 加入圖層控制器 (這會自動產生 Checkbox) ---
    folium.LayerControl().add_to(m)

    m.save(output_html)
    print(f"    - ✅ 地圖已儲存：{output_html}")


if __name__ == "__main__":
    # 🔹設定資料夾路徑
    gpx_folder = Path("hikingbook_route")
    txt_folder = Path("txt_data")
    gpx_output_folder = Path("gpx_data")
    csv_output_folder = Path("csv_data")
    html_output_folder = Path("Html_data")

    # 🔹建立輸出資料夾（若不存在）
    gpx_output_folder.mkdir(exist_ok=True)
    csv_output_folder.mkdir(exist_ok=True)
    html_output_folder.mkdir(exist_ok=True)
    
    print("--- 開始批次處理 ---")
    print(f"🔍 目前工作目錄: {os.getcwd()}")
    print(f"📂 正在從 '{gpx_folder.resolve()}' 讀取 GPX 檔案")
    print(f"📂 正在從 '{txt_folder.resolve()}' 讀取 TXT 檔案")
    print("-" * 40)

    # 🔹 尋找所有 .gpx 檔案
    gpx_files = list(gpx_folder.glob("*.gpx"))

    if not gpx_files:
        print("❌ 錯誤：在 'hikingbook_route' 資料夾中找不到任何 .gpx 檔案。")
        print("   請確認：")
        print("   1. 'hikingbook_route' 資料夾與您的 Python 腳本位於同一層級。")
        print("   2. 資料夾內確實有 .gpx 結尾的檔案。")
    else:
        print(f"✅ 找到 {len(gpx_files)} 個 GPX 檔案，準備開始處理...\n")

    # 🔹迴圈處理每個 GPX 檔案
    for gpx_file in gpx_files:
        base_name = gpx_file.stem  # 檔名不含副檔名
        txt_file = txt_folder / f"{base_name}.txt"

        print(f"🔸 正在處理: {gpx_file.name}")
        print(f"   - 正在尋找對應的 TXT: {txt_file.name}")

        if not txt_file.exists():
            print(f"   - ⚠️ 警告: 找不到對應的 TXT 檔案，略過此 GPX 檔案。")
            print("-" * 40)
            continue
        
        print(f"   - ✓ 找到對應的 TXT 檔案。")

        # 1. 去除時間
        gpx_obj = remove_time(str(gpx_file))

        # 2. 插入通訊點
        comm_pts = load_communication_points(str(txt_file))
        if not comm_pts:
            print(f"   - ⚠️ 警告: 未能從 {txt_file.name} 讀取任何通訊點，略過此檔案。")
            print("-" * 40)
            continue
            
        gpx_with_pts = insert_points(gpx_obj, comm_pts)

        # 3. 切出路段
        start_latlon = (comm_pts[0][1], comm_pts[0][2])
        end_latlon = (comm_pts[-1][1], comm_pts[-1][2])
        sliced_gpx = split_single_segment(gpx_with_pts, start_latlon, end_latlon)

        # 4. 匯出 GPX + CSV
        gpx_outfile = gpx_output_folder / f"{base_name}_processed.gpx"
        csv_outfile = csv_output_folder / f"{base_name}_processed.csv"
        export_gpx_and_csv(sliced_gpx, str(gpx_outfile), str(csv_outfile))

        # 5. 儲存地圖 HTML
        html_outfile = html_output_folder / f"{base_name}_map.html"
        visualize_with_checkbox(gpx_with_pts, sliced_gpx, comm_pts, str(html_outfile))
        
        print("-" * 40)

    print("\n🏁 批次處理完成")


--- 開始批次處理 ---
🔍 目前工作目錄: c:\資展\專題\特徵\標準桃山
📂 正在從 'C:\資展\專題\特徵\標準桃山\hikingbook_route' 讀取 GPX 檔案
📂 正在從 'C:\資展\專題\特徵\標準桃山\txt_data' 讀取 TXT 檔案
----------------------------------------
❌ 錯誤：在 'hikingbook_route' 資料夾中找不到任何 .gpx 檔案。
   請確認：
   1. 'hikingbook_route' 資料夾與您的 Python 腳本位於同一層級。
   2. 資料夾內確實有 .gpx 結尾的檔案。

🏁 批次處理完成
